In [ ]:
# парсим простой сайт

In [33]:
from tqdm import tqdm_notebook
from pyquery import PyQuery as pq
import requests

response_poems = requests.get("https://www.stihi.ru/2019/03/28/5717")
# следующей строкой можно вывести первые 500 строчек сохраненного html-документа
# response_poems.text[:500]

# автор
author = pq(response_poems.text).find("div.titleauthor").text()
# на следующей строке открываем имя автора
# author

# как оно называется
title = pq(response_poems.text).find("h1").text()
#title

# ну и сам текст стиха
text = pq(response_poems.text).find("div.text").text()
#text

# теперь попробуем скачать все стихотворение с 1 странички произведений раздела "мистика и эзотерика"
# сначала соберем ссылочки на сами стихи и запишем их в отдельный лист
response_poems_full = requests.get("https://www.stihi.ru/poems/list.html?topic=13")
poems_urls = []

for poem in pq(response_poems_full.text).find("a.poemlink"):
    url = pq(poem).attr("href")
    poems_urls.append("https://www.stihi.ru" + url)
    
    
# оборачивает в цикл, которым записываем текст стиха, название и автора с каждой ссылки в отдельные файлики в корневом каталоге
for num, poem_url in tqdm_notebook(enumerate(poems_urls)):
    res = requests.get(poem_url)
    
    title = pq(res.text).find("h1").text()
    author = pq(res.text).find("div.titleauthor").text()
    
     text = ""
    for poem in pq(res.text).find("div.text"):
        text += pq(poem).text().replace("\n", "\n")
    
    try:
        with open(f"stihi/{num}.txt", "wt", encoding="utf-8") as f:
            f.write(title + "\n" + author + "\n" + text)
    except UnicodeEncodeError as err:
        print(err, poem_url)

In [ ]:
# загрузка с динамического сайта (медуза, например)

In [39]:
# сначала надо найти, откуда подгружаются статьи
url_t = "https://meduza.io/api/w4/search?chrono=news&page={page}&per_page=24&locale=ru"


# выгрузим статьи с первых 10 страниц
# сначала соберем ссылки на эти странички (код - нетворк - превью)
articles = []
for page in range(1, 11):
    url = url_t.format(page = page)
    print(url)
    res = requests.get(url)
    articles.extend(res.json()["collection"])
    
# количество статеек, которые мы соберем
# len(articles)

# как выглядят сами ссылки на статьи
# articles[0]

# соберем тексты этих статей
for num, article_url in tqdm_notebook(enumerate(articles)):
    res = requests.get("https://meduza.io/" + article_url)

    title = ""
    for title in pq(res.text).find("h1.SimpleTitle-root"):
        title = pq(title).text().replace("\xa0", " ")

    text = ""
    for paragraph in pq(res.text).find("div.GeneralMaterial-article p"):
        text += pq(paragraph).text().replace("\xa0", " ")
    try:
        with open(f"texts_meduza/{num}.txt", "wt", encoding="utf-8") as f:
            f.write(title + "\n" + text)
    except UnicodeEncodeError as err:
        print(err, article_url)
        

# можно также дособрать к ним метаинформацию, если друг что-то важно для анализа. например, добавим дату публикации

for num, article_url in tqdm_notebook(enumerate(articles)):
    res = requests.get("https://meduza.io/" + article_url)

    title = ""
    for title in pq(res.text).find("h1.SimpleTitle-root"):
        title = pq(title).text().replace("\xa0", " ")
    time = pq(res.text).find("time.Timestamp-root").text()
    
    try:
        with open(f"texts_meduza_meta/{num}.txt", "wt", encoding="utf-8") as f:
            f.write(title + "\n" + time)
    except UnicodeEncodeError as err:
        print(err, article_url)

https://meduza.io/api/w4/search?chrono=news&page=7&per_page=24&locale=ru
https://meduza.io/api/w4/search?chrono=news&page=7&per_page=24&locale=ru
https://meduza.io/api/w4/search?chrono=news&page=7&per_page=24&locale=ru
https://meduza.io/api/w4/search?chrono=news&page=7&per_page=24&locale=ru
https://meduza.io/api/w4/search?chrono=news&page=7&per_page=24&locale=ru
https://meduza.io/api/w4/search?chrono=news&page=7&per_page=24&locale=ru
https://meduza.io/api/w4/search?chrono=news&page=7&per_page=24&locale=ru
https://meduza.io/api/w4/search?chrono=news&page=7&per_page=24&locale=ru
https://meduza.io/api/w4/search?chrono=news&page=7&per_page=24&locale=ru
https://meduza.io/api/w4/search?chrono=news&page=7&per_page=24&locale=ru
